# 1. Introduction

# 2. Description of the Dataset

# 3. Python Libraries and Modules

# 3. Data Precessing and Cleaning

## Data Types of Each Columns

## Null Columns

## Duplicate Entries

# 5. Exploratory Data Analysis

## 1. What is the distribution of household sizes (PUFHHSIZE)?

In [ ]:
## 2. What is the distribution of age (PUFC05_AGE)?

## 3. What is the distribution of regions (PUFREG)?

## 4. What is the distribution of provinces (PUFPRV)?

## 5. What is the ratio of males vs. females (PUFC04_SEX)?

## 6. What are the most common education levels (PUFC07_GRADE)?

## 7. What are the most common occupations (PUFC14_PROCC)?

## 8. What are the most common industries (PUFC16_PKB)?

## 9. How do daily working hours differ by occupation? (PUFC14_PROCC, PUFC18_PNWHRS)?

In [ ]:
## 10. What are the top 5 occupations (PUFC07_GRADE) per nature of employment (PUFC14_PROCC)?

# Resampling

# Scaling

# Splitting the Dataset into training, testing, and validation

# Machine Learning

## Initial Model Training

## Error Analysis

## Improving Model Performance

# Insights and Conclusion

# References